__Dependencies__

In [1]:
import copy  # copy big/deep objects by value
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import os  # OS operations (read/write files/folders)
import warnings  # hide warnings
# process parallelization
from multiprocessing import Lock, Manager, Pool, RawArray, cpu_count, managers

import matplotlib.patches as mpatches  # create custom plot legends
import matplotlib.pyplot as plt  # mother of plots for Python
import matplotlib.ticker as ticker  # matplotlib ticker utils
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes
import seaborn as sns  # matplotlib plotting nice with shortcuts
from IPython.display import display  # print nicely
from matplotlib.lines import Line2D  # matplotlib custom lines
from scipy import stats  # statistical tests
from tqdm.notebook import tqdm  # mother of progressbars for Python

# from matplotlib.ticker import FormatStrFormatter  # tick formatter

__Options and constants__

In [2]:
warnings.filterwarnings("ignore")
# set default float display format to 2 decimals
pd.options.display.float_format = "{:.3f}".format

PATH_PROC = "./data/processed"

# progress bar customized format
B_FORMAT = """📄 {n_fmt} of {total_fmt} {desc} processed: {bar} 
            {percentage:3.0f}% ⏱️{elapsed} ⏳{remaining} ⚙️{rate_fmt}{postfix}"""

"""
Event1: 1162-1400
Event2: 1958-2080
Event3: 2596-2890
"""
event_ranges = [(1162, 1400), (1958, 2080), (2596, 2890)]

CORES = cpu_count()  # number of cpu threads for multiprocessing
print(f"Total CPU threads: {CORES}")

Total CPU threads: 16


__Helper functions__

In [3]:
def pbar_fork_hack(l=None):
    """
    Hack to enforce progress bars to be displayed by fork processes on
    IPython Apps like Jupyter Notebooks.

    Avoids [IPKernelApp] WARNING | WARNING: attempted to send message from fork

    Important: pass this function as argument for the initializer parameter
    while initializing a multiprocessing pool to make it work. E.g.:

    pool = Pool(processes=N_CORES, initializer=pbar_fork_hack)

    Source:
     - https://github.com/ipython/ipython/issues/11049#issue-306086846
     - https://github.com/tqdm/tqdm/issues/485#issuecomment-473338308
    """
    if l:
        global lock
        lock = l
    print(" ", end="", flush=True)


def angle(a, b):
    """
    Angle (in degrees) between two vectors.

    Given two 3D vectors (a and b):
    a = [xa, ya, za] , b = [xb, yb, zb]

    And the basic geometric formula for the dot product:
    a · b = |a| * |b| * cos(α) => α = arccos[(a · b) / (|a| * |b|)]

    In other words:
    angle = arccos[(xa * xb + ya * yb + za * zb) / (√(xa2 + ya2 + za2) * √(xb2 + yb2 + zb2))]

    Source: https://www.omnicalculator.com/math/angle-between-two-vectors

    Parameters:
        a (numpy.array): First vector
        b (numpy.array): Second vector
    Returns:
        float: Angle between vectors a and b in degrees.
    """
    # a · b
    dotp = np.dot(a, b)

    # magnitude vectors |a| and |b|
    m_a = np.linalg.norm(a)
    m_b = np.linalg.norm(b)

    # |a| * |b|
    m_prod = m_a * m_b

    # (a * b) / (|a| * |b|)
    res = dotp / m_prod

    # α = arccos[(a · b) / (|a| * |b|)]
    an = np.arccos(res)

    # calculated angle from radians to degrees
    return np.degrees(an)

__Read participants cleaned data__

In [4]:
parts = pd.read_csv("./participants_clean.csv")
parts = parts.set_index("id")  # set uid column as index (remove default)
parts

,date,expo,side,condition,nulls_%,sky_%,insidecar_%,questionnaire
id,,,,,,,,
4f583872ffed48249874b00f3d389cfc,2019-05-14 13:21:00,MSW,right,TaxiDriver,1.610,13.740,0.487,No
33d68ad55ef544fab9fd08108e3913ca,2019-05-14 16:44:00,MSW,right,AVAS,0.037,0.674,0.000,No
8598a83e2af441b8bcd0ae5d84beb875,2019-05-14 16:51:00,MSW,right,RadioTalk,0.037,33.134,0.000,No
37d7e67934974217830bb429bba7fd76,2019-05-14 16:54:00,MSW,left,AVAS,0.824,18.607,0.374,No
42bac596059749b5b8e8e83ae61de9b4,2019-05-14 16:56:00,MSW,right,TaxiDriver,0.225,26.994,0.000,No
...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,2019-12-30 15:29:00,BMBF,right,AVAS,0.300,32.160,0.000,No
611d35a7c3e940cc82495e53c2a8532d,2020-01-03 16:12:00,BMBF,right,TaxiDriver,0.674,15.462,0.037,No
3b6fda285d9e412eb081986b2f22a2e3,2020-01-03 16:13:00,BMBF,left,AVAS,1.797,3.370,0.150,No


Collect all gazes and hit frames on critical events 

In [5]:
def events_frames_all(uid):
    """
    Collect critical event frames of all participants.
    'h_start' -> 1st frame number, all hits
    'g_start' -> 1st frame number, all gazes
    'h_all'   -> All frames, all hits
    'g_all'   -> All frames, all gazes

    Parameters:
        uid (str): Participant unique identifier.
    """
    # load event data
    events = pd.read_csv(f"{PATH_PROC}/hit_events/{uid}.csv")
    for objct in objcts:
        # select all hits of the current event
        query = events["name"].str.contains(objct, regex=False, na=False)
        h_events = events[query]
        starts = h_events.start.tolist()
        ends = h_events.end.tolist()
        # store list of start hits (any)
        if len(starts) > 0:
            # store hit starts
            for f in starts:
                lock.acquire()
                res["h_start"][objct].append(f)
                lock.release()
            # collect all hits
            frames = [
                [i for i in range(s, e + 1)] for s, e in zip(starts, ends)
            ]
            # flatten list
            frames = [h for hlist in frames for h in hlist]
            # store all hits
            for f in frames:
                lock.acquire()
                res["h_all"][objct].append(f)
                lock.release()
            # select only gaze events
            h_events = h_events[h_events["type"] == "gaze"]
            starts = h_events.start.tolist()
            ends = h_events.end.tolist()
            # store list of start hits (gazes)
            for f in starts:
                lock.acquire()
                res["g_start"][objct].append(f)
                lock.release()
            # collect all hits
            frames = [
                [i for i in range(s, e + 1)] for s, e in zip(starts, ends)
            ]
            # flatten list
            frames = [h for hlist in frames for h in hlist]
            # store all gaze hits
            for f in frames:
                lock.acquire()
                res["g_all"][objct].append(f)
                lock.release()


manager = Manager()  # manage shared memory types

# event names and hit/gaze groups to build dict
objcts = ["jogging_man", "fast_car", "walking_man"]
h_groups = ["h_start", "h_all", "g_start", "g_all"]

# proxy object progress bar
proxy_pb = tqdm(
    iterable=h_groups,
    total=len(h_groups),
    desc="📂 proxy objects",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

# convert into memory-shared dict of dicts for multiprocessing
res = manager.dict(
    {g: manager.dict({o: manager.list() for o in objcts}) for g in proxy_pb}
)

l = Lock()

# initialize pool of processes according to the available cpu core threads
pool = Pool(processes=CORES, initializer=pbar_fork_hack, initargs=(l,))

# participants ids
uids = parts.index.tolist()

# participants progress bar
parts_pbar = tqdm(
    iterable=pool.imap_unordered(func=events_frames_all, iterable=uids),
    total=len(uids),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

# loop necessary for displaying properly the progressbar with multiprocessing
# source: https://stackoverflow.com/a/40133278
for _ in parts_pbar:
    pass

# close pool instance, no more work to submit
pool.close()
# wait for the worker processes to terminate
pool.join()

📄 0 of 4 📂 proxy objects processed:                                                                           …

📄 0 of 24356 📂 participants processed:                                                                        …

Make proxy object readable and store it as a JSON dict

In [6]:
# decode proxy object
# we include the filter (e < 2000 and c == "walking_man") since due to
# Farbod's NPC's random respawn, the walking man appeared during the
# instructions of the experiment and was hit
temp = dict(res)
temp = {k: dict(v) for k, v in temp.items()}
temp = {
    k: {
        c: [e for e in list(b) if not (e < 2000 and c == "walking_man")]
        for c, b in v.items()
    }
    for k, v in temp.items()
}

# store it
with open("gazes_and_hits-event_frames.json", "w") as file:
    json.dump(temp, file)

Collect gazes and hit frames on critical events per participant (individual files)

In [7]:
def events_frames_part(uid):
    """
    Collect and store critical event frames of each participant.
    'h_start' -> 1st frame number, all hits
    'g_start' -> 1st frame number, all gazes
    'h_all'   -> All frames, all hits
    'g_all'   -> All frames, all gazes

    Parameters:
        uid (str): Participant unique identifier.
    """
    # dict to collect hit and gaze frames
    res = {o: {g: [] for g in h_groups} for o in objcts}
    # load event data
    events = pd.read_csv(f"{PATH_PROC}/hit_events/{uid}.csv")
    for objct in objcts:
        # select all hits of the current event
        query = events["name"].str.contains(objct, regex=False, na=False)
        h_events = events[query]
        starts = h_events.start.tolist()
        ends = h_events.end.tolist()
        # store list of start hits (any)
        if len(starts) > 0:
            # store hit starts
            for f in starts:
                res[objct]["h_start"].append(f)
            # collect all hits
            frames = [
                [i for i in range(s, e + 1)] for s, e in zip(starts, ends)
            ]
            # flatten list
            frames = [h for hlist in frames for h in hlist]
            # store all hits
            for f in frames:
                res[objct]["h_all"].append(f)
            # select only gaze events
            h_events = h_events[h_events["type"] == "gaze"]
            starts = h_events.start.tolist()
            ends = h_events.end.tolist()
            # store list of start hits (gazes)
            for f in starts:
                res[objct]["g_start"].append(f)
            # collect all hits
            frames = [
                [i for i in range(s, e + 1)] for s, e in zip(starts, ends)
            ]
            # flatten list
            frames = [h for hlist in frames for h in hlist]
            # store all gaze hits
            for f in frames:
                res[objct]["g_all"].append(f)

            # store results
            fname = f"{PATH_PROC}/events/frames/{uid}.json"
            with open(fname, "w") as file:
                json.dump(res, file)


# event names and hit/gaze groups to build dict
objcts = ["jogging_man", "fast_car", "walking_man"]
h_groups = ["h_start", "h_all", "g_start", "g_all"]

# participants ids
uids = parts.index.tolist()

# initialize pool of processes according to the available cpu core threads
pool = Pool(processes=CORES, initializer=pbar_fork_hack)

# participants progress bar
parts_pbar = tqdm(
    iterable=pool.imap_unordered(func=events_frames_part, iterable=uids),
    total=len(uids),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

# loop necessary for displaying properly the progressbar with multiprocessing
# source: https://stackoverflow.com/a/40133278
for _ in parts_pbar:
    pass

# close pool instance, no more work to submit
pool.close()
# wait for the worker processes to terminate
pool.join()

📄 0 of 24356 📂 participants processed:                                                                        …

Compute gaze vs rest stats and store gaze length and gaze counts (events)

In [9]:
def gazes_vs_rest(uid):
    """
    Compute gaze vs rest stats and store gaze length and gaze counts (events)
    of a participant given its UID.

    Parameters:
        uid (str): Participant unique identifier.
    """
    h_events = pd.read_csv(f"{PATH_PROC}/hit_events/{uid}.csv")
    # filter out Sky events since they are uninformative of gaze/rest
    h_events = h_events[h_events.group != "Sky"]
    total = h_events.index.size

    # disentangle gazes and rest events
    rest = h_events[h_events["type"] == "rest"]
    gazes = h_events[h_events["type"] == "gaze"]

    # compute length (time in s) and event counts
    total_r = rest.index.size
    total_g = gazes.index.size
    rest_len = rest["length"].sum()
    gaze_len = gazes["length"].sum()
    total_len = rest_len + gaze_len

    # store stats
    hev_cnts[uid]["rest_cnt"] = total_r
    hev_cnts[uid]["gaze_cnt"] = total_g
    hev_cnts[uid]["rest_avg"] = rest["length"].mean()
    hev_cnts[uid]["gaze_avg"] = gazes["length"].mean()
    hev_cnts[uid]["rest_mdn"] = rest["length"].median()
    hev_cnts[uid]["gaze_mdn"] = gazes["length"].median()
    hev_cnts[uid]["rest_dist_avg"] = rest["distance"].mean()
    hev_cnts[uid]["gaze_dist_avg"] = gazes["distance"].mean()
    hev_cnts[uid]["rest_dist_mdn"] = rest["distance"].median()
    hev_cnts[uid]["gaze_dist_mdn"] = gazes["distance"].median()
    hev_cnts[uid]["rest_len"] = rest_len
    hev_cnts[uid]["gaze_len"] = gaze_len
    hev_cnts[uid]["rest_l_%"] = rest_len * 100 / total_len
    hev_cnts[uid]["gaze_l_%"] = gaze_len * 100 / total_len
    hev_cnts[uid]["rest_c_%"] = total_r * 100 / total
    hev_cnts[uid]["gaze_c_%"] = total_g * 100 / total

    # iterate over objects groups for current participant
    grps = gazes.group.unique()
    for grp in grps:
        # gazes for the current group, compute and store lengths and counts
        sel = gazes[gazes.group == grp]
        g_len = sel["length"].sum()
        g_cnt = sel.index.size
        grp_lpercs[uid][grp] = g_len * 100 / gaze_len
        grp_cpercs[uid][grp] = g_cnt * 100 / total_g
        grp_len[uid][grp] = g_len
        grp_cnt[uid][grp] = g_cnt
    # iterate over all groups possible
    for grp in grp_cats:
        # if the group is not current participant, add 0 counter
        if not grp_lpercs[uid][grp]:
            grp_lpercs[uid][grp] = 0
            grp_cpercs[uid][grp] = 0
            grp_len[uid][grp] = 0
            grp_cnt[uid][grp] = 0


# participants ids
uids = parts.index.tolist()

cats = [
    "rest_cnt",
    "gaze_cnt",
    "rest_avg",
    "gaze_avg",
    "rest_mdn",
    "gaze_mdn",
    "rest_dist_avg",
    "gaze_dist_avg",
    "rest_dist_mdn",
    "gaze_dist_mdn",
    "rest_len",
    "gaze_len",
    "rest_l_%",
    "gaze_l_%",
    "rest_c_%",
    "gaze_c_%",
]
stats = {cat: None for cat in cats}

grp_cats = [
    "Buildings",
    "Dynamic Cars",
    "Dynamic Pedestrians",
    "Inside Car",
    "Nature",
    "Roads",
    # "Sky",
    "Static Cars",
    "Static Pedestrians",
    "Street Objects",
    "Traffic Lights",
    "Traffic Signs",
]
grp_stats = {cat: None for cat in grp_cats}

manager = Manager()  # manage shared memory types

# convert into memory-shared dict of dicts for multiprocessing
hev_cnts = manager.dict({uid: manager.dict(stats) for uid in uids})
grp_lpercs = manager.dict({uid: manager.dict(grp_stats) for uid in uids})
grp_cpercs = manager.dict({uid: manager.dict(grp_stats) for uid in uids})
grp_len = manager.dict({uid: manager.dict(grp_stats) for uid in uids})
grp_cnt = manager.dict({uid: manager.dict(grp_stats) for uid in uids})

# initialize pool of processes according to the available cpu core threads
pool = Pool(processes=CORES, initializer=pbar_fork_hack)

# participants progress bar
parts_pbar = tqdm(
    iterable=pool.imap_unordered(func=gazes_vs_rest, iterable=uids),
    total=len(uids),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

# loop necessary for displaying properly the progressbar with multiprocessing
# source: https://stackoverflow.com/a/40133278
for _ in parts_pbar:
    pass

# close pool instance, no more work to submit
pool.close()
# wait for the worker processes to terminate
pool.join()

# parse and store collected data
hev_cnts = dict(hev_cnts)
hev_cnts = {k: dict(v) for k, v in hev_cnts.items()}
hev_cnts = pd.DataFrame(hev_cnts).transpose()
hev_cnts.to_csv("./hit_events_counters.csv")
display(hev_cnts)

grp_lpercs = dict(grp_lpercs)
grp_cpercs = dict(grp_cpercs)
grp_len = dict(grp_len)
grp_cnt = dict(grp_cnt)
grp_lpercs = {k: dict(v) for k, v in grp_lpercs.items()}
grp_cpercs = {k: dict(v) for k, v in grp_cpercs.items()}
grp_len = {k: dict(v) for k, v in grp_len.items()}
grp_cnt = {k: dict(v) for k, v in grp_cnt.items()}
grp_lpercs = pd.DataFrame(grp_lpercs).transpose()
grp_cpercs = pd.DataFrame(grp_cpercs).transpose()
grp_len = pd.DataFrame(grp_len).transpose()
grp_cnt = pd.DataFrame(grp_cnt).transpose()
grp_lpercs.to_csv("./group_length_percentages.csv")
grp_cpercs.to_csv("./group_counts_percentages.csv")
grp_len.to_csv("./group_lengths.csv")
grp_cnt.to_csv("./group_counts.csv")
display(grp_lpercs)
display(grp_cpercs)
display(grp_len)
display(grp_cnt)

📄 0 of 24356 📂 participants processed:                                                                        …

,rest_cnt,gaze_cnt,rest_avg,gaze_avg,rest_mdn,gaze_mdn,rest_dist_avg,gaze_dist_avg,rest_dist_mdn,gaze_dist_mdn,rest_len,gaze_len,rest_l_%,gaze_l_%,rest_c_%,gaze_c_%
4f583872ffed48249874b00f3d389cfc,434.000,67.000,0.066,0.876,0.033,0.495,104.681,100.493,85.326,66.196,28.644,58.707,32.792,67.208,86.627,13.373
33d68ad55ef544fab9fd08108e3913ca,280.000,80.000,0.083,0.811,0.066,0.611,20.270,10.064,2.389,2.369,23.232,64.911,26.357,73.643,77.778,22.222
8598a83e2af441b8bcd0ae5d84beb875,110.000,37.000,0.089,1.336,0.066,0.660,95.261,148.664,47.726,92.824,9.834,49.434,16.592,83.408,74.830,25.170
37d7e67934974217830bb429bba7fd76,586.000,69.000,0.059,0.767,0.033,0.396,120.903,101.458,81.686,58.163,34.815,52.899,39.691,60.309,89.466,10.534
42bac596059749b5b8e8e83ae61de9b4,333.000,70.000,0.073,0.774,0.066,0.462,166.052,158.886,104.916,100.255,24.420,54.186,31.066,68.934,82.630,17.370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,197.000,67.000,0.080,0.786,0.066,0.462,155.156,120.764,74.446,73.856,15.675,52.635,22.947,77.053,74.621,25.379
611d35a7c3e940cc82495e53c2a8532d,380.000,56.000,0.070,1.097,0.033,0.627,107.759,69.995,69.104,45.891,26.532,61.446,30.158,69.842,87.156,12.844
3b6fda285d9e412eb081986b2f22a2e3,456.000,85.000,0.068,0.665,0.033,0.495,77.189,41.573,36.823,14.536,30.855,56.496,35.323,64.677,84.288,15.712
18ffb0abdc8642098c479380bfa533d1,544.000,77.000,0.068,0.589,0.033,0.429,117.366,85.315,65.854,42.028,37.092,45.342,44.996,55.004,87.601,12.399


,Buildings,Dynamic Cars,Dynamic Pedestrians,Inside Car,Nature,Roads,Static Cars,Static Pedestrians,Street Objects,Traffic Lights,Traffic Signs
4f583872ffed48249874b00f3d389cfc,31.366,1.574,1.911,3.204,6.127,31.141,0.000,0.000,1.967,22.709,0.000
33d68ad55ef544fab9fd08108e3913ca,0.000,0.915,0.000,44.179,0.458,46.772,0.000,0.000,0.559,7.117,0.000
8598a83e2af441b8bcd0ae5d84beb875,77.503,0.000,0.000,1.268,1.869,0.000,0.000,0.000,0.734,18.625,0.000
37d7e67934974217830bb429bba7fd76,32.626,4.741,2.121,8.921,9.545,2.121,0.624,0.000,3.805,35.496,0.000
42bac596059749b5b8e8e83ae61de9b4,63.825,0.000,0.000,0.974,11.815,0.000,0.000,0.000,4.263,19.123,0.000
...,...,...,...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,64.702,0.000,0.000,12.226,5.204,0.000,0.000,0.000,1.944,15.925,0.000
611d35a7c3e940cc82495e53c2a8532d,30.988,0.000,0.000,39.527,7.626,4.619,0.000,0.000,1.719,14.930,0.591
3b6fda285d9e412eb081986b2f22a2e3,13.843,0.000,0.818,32.535,1.168,29.381,0.000,1.986,10.397,9.871,0.000
18ffb0abdc8642098c479380bfa533d1,32.460,0.582,0.000,12.082,10.917,14.119,0.000,0.000,3.275,25.182,1.383


,Buildings,Dynamic Cars,Dynamic Pedestrians,Inside Car,Nature,Roads,Static Cars,Static Pedestrians,Street Objects,Traffic Lights,Traffic Signs
4f583872ffed48249874b00f3d389cfc,34.328,2.985,1.493,4.478,11.940,28.358,0.000,0.000,5.970,10.448,0.000
33d68ad55ef544fab9fd08108e3913ca,0.000,2.500,0.000,52.500,1.250,40.000,0.000,0.000,1.250,2.500,0.000
8598a83e2af441b8bcd0ae5d84beb875,64.865,0.000,0.000,5.405,8.108,0.000,0.000,0.000,2.703,18.919,0.000
37d7e67934974217830bb429bba7fd76,37.681,4.348,1.449,14.493,15.942,5.797,1.449,0.000,8.696,10.145,0.000
42bac596059749b5b8e8e83ae61de9b4,57.143,0.000,0.000,2.857,20.000,0.000,0.000,0.000,7.143,12.857,0.000
...,...,...,...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,62.687,0.000,0.000,13.433,8.955,0.000,0.000,0.000,2.985,11.940,0.000
611d35a7c3e940cc82495e53c2a8532d,33.929,0.000,0.000,32.143,14.286,7.143,0.000,0.000,5.357,5.357,1.786
3b6fda285d9e412eb081986b2f22a2e3,11.765,0.000,1.176,30.588,2.353,34.118,0.000,2.353,10.588,7.059,0.000
18ffb0abdc8642098c479380bfa533d1,31.169,1.299,0.000,16.883,15.584,12.987,0.000,0.000,5.195,14.286,2.597


,Buildings,Dynamic Cars,Dynamic Pedestrians,Inside Car,Nature,Roads,Static Cars,Static Pedestrians,Street Objects,Traffic Lights,Traffic Signs
4f583872ffed48249874b00f3d389cfc,18.414,0.924,1.122,1.881,3.597,18.282,0.000,0.000,1.155,13.332,0.000
33d68ad55ef544fab9fd08108e3913ca,0.000,0.594,0.000,28.677,0.297,30.360,0.000,0.000,0.363,4.620,0.000
8598a83e2af441b8bcd0ae5d84beb875,38.313,0.000,0.000,0.627,0.924,0.000,0.000,0.000,0.363,9.207,0.000
37d7e67934974217830bb429bba7fd76,17.259,2.508,1.122,4.719,5.049,1.122,0.330,0.000,2.013,18.777,0.000
42bac596059749b5b8e8e83ae61de9b4,34.584,0.000,0.000,0.528,6.402,0.000,0.000,0.000,2.310,10.362,0.000
...,...,...,...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,34.056,0.000,0.000,6.435,2.739,0.000,0.000,0.000,1.023,8.382,0.000
611d35a7c3e940cc82495e53c2a8532d,19.041,0.000,0.000,24.288,4.686,2.838,0.000,0.000,1.056,9.174,0.363
3b6fda285d9e412eb081986b2f22a2e3,7.821,0.000,0.462,18.381,0.660,16.599,0.000,1.122,5.874,5.577,0.000
18ffb0abdc8642098c479380bfa533d1,14.718,0.264,0.000,5.478,4.950,6.402,0.000,0.000,1.485,11.418,0.627


,Buildings,Dynamic Cars,Dynamic Pedestrians,Inside Car,Nature,Roads,Static Cars,Static Pedestrians,Street Objects,Traffic Lights,Traffic Signs
4f583872ffed48249874b00f3d389cfc,23,2,1,3,8,19,0,0,4,7,0
33d68ad55ef544fab9fd08108e3913ca,0,2,0,42,1,32,0,0,1,2,0
8598a83e2af441b8bcd0ae5d84beb875,24,0,0,2,3,0,0,0,1,7,0
37d7e67934974217830bb429bba7fd76,26,3,1,10,11,4,1,0,6,7,0
42bac596059749b5b8e8e83ae61de9b4,40,0,0,2,14,0,0,0,5,9,0
...,...,...,...,...,...,...,...,...,...,...,...
cfe9482181f74f80b88cd4b1c048ab94,42,0,0,9,6,0,0,0,2,8,0
611d35a7c3e940cc82495e53c2a8532d,19,0,0,18,8,4,0,0,3,3,1
3b6fda285d9e412eb081986b2f22a2e3,10,0,1,26,2,29,0,2,9,6,0
18ffb0abdc8642098c479380bfa533d1,24,1,0,13,12,10,0,0,4,11,2
